<a href="https://colab.research.google.com/github/kameoman/pythonlesson/blob/main/%E5%8F%8E%E7%A9%AB%E9%87%8F%E4%BA%88%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pystan
!pip install fbprophet

In [9]:
import pandas as pd
from fbprophet import Prophet

In [14]:
_df = pd.read_csv('/content/syukaku_syunka_tomato_20.csv', encoding="cp932")

In [15]:
_df.head()

,(10) 夏秋トマト,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,全 国\n・\n都 道 府 県,作付面積,10ａ当たり\n収 量,収穫量,出荷量
1,NaN,ha,㎏,ｔ,ｔ
2,全国,7550,4260,321300,276200
3,（都道府県）,NaN,NaN,NaN,NaN
4,北海道,718,7760,55700,51300
